In [30]:
import random
import math
import copy
import time

file_path = "../Instances/c103.txt" 


In [31]:
class Node:
    def __init__(self, num, x, y, service_time, score, open_time, close_time):
        self.num = num
        self.x = x
        self.y = y
        self.service_time = service_time
        self.open_time = open_time
        self.close_time = close_time
        self.score = score


def is_feasible (tour, source):
    t_current=0
    t_max=source.close_time
    current_vertex=source
    # print("tour", tour)
    for i in range(len(tour)+1):
        next_vertex= None
        if i==len(tour):
            next_vertex= source
        else:
            next_vertex=tour[i]
        if max(t_current+math.dist([next_vertex.x,next_vertex.y],[current_vertex.x,current_vertex.y]),next_vertex.open_time)+next_vertex.service_time +math.dist([next_vertex.x,next_vertex.y],[source.x,source.y])>t_max:
            return False
        if (t_current + math.dist([next_vertex.x,next_vertex.y],[current_vertex.x,current_vertex.y]) <= next_vertex.close_time):
                t_current = max(t_current +math.dist([next_vertex.x,next_vertex.y],[current_vertex.x,current_vertex.y]), next_vertex.open_time) + next_vertex.service_time;
                current_vertex = next_vertex
            
        else: 
            return False
    return True

def calculate_score(solution, source):
    score=0
    for i in range(len(solution)):
        tour=solution[i]
        if is_feasible(tour, source):
            for j in range (len(tour)):
                score += tour[j].score
    return score

def solution_to_num(solution):
    solution_in_num = [[obj.num for obj in inner_list] for inner_list in solution]
    return solution_in_num

def tour_to_num(tour):
    tour_in_num = [obj.num for obj in tour]
    return tour_in_num

In [32]:
def euclidian_distance(first_node: Node, second_node: Node):
    return math.dist([first_node.x, first_node.y], [second_node.x, second_node.y])

def random_initial_solution(nodes, team_number):
    solution = []
    candidate = copy.deepcopy(nodes)
    source = nodes[0]
    t_max = source.close_time
    candidate.pop(0)
    
    for i in range(team_number):
        t_current = 0.0
        tour = []
        current_vertex = source
        counter = 0
        
        while len(candidate) > 0:
            random_candidate = random.randint(0, len(candidate) - 1)
            next_vertex = candidate[random_candidate]
            
            if (max(t_current + euclidian_distance(next_vertex, current_vertex), next_vertex.open_time) +
                    next_vertex.service_time + euclidian_distance(next_vertex, source)) > t_max:
                break
            
            if t_current + euclidian_distance(next_vertex, current_vertex) <= next_vertex.close_time:
                t_current = max(t_current + euclidian_distance(next_vertex, current_vertex),
                                next_vertex.open_time) + next_vertex.service_time
                tour.append(next_vertex)
                current_vertex = next_vertex
                candidate.pop(random_candidate)
            else:
                if counter == 30:
                    break
                else:
                    counter += 1
                    continue
        
        solution.append(tour)
    
    return solution


In [33]:



lines = []

with open(file_path, 'r') as file:
    for line in file:
        lines.append(line.rstrip())


k, v, N, t = map(int, lines[0].split())
# v = 3
nodes = []
optimal_score = 0
for i, line in enumerate(lines[2:]):
    line_list = line.split()
    num = int(line_list[0])
    x = float(line_list[1])
    y = float(line_list[2])
    service_time = float(line_list[3])
    score = float(line_list[4])
    open_time = int(line_list[-2]) if lines.index(line) != 2 else 0
    close_time = int(line_list[-1]) if lines.index(line) != 2 else int(line_list[-1])
    node = Node(num, x, y, service_time, score, open_time, close_time)
    nodes.append(node)
    optimal_score+=score
source = nodes[0]
optimal_score


1810.0

In [34]:
N

100

In [35]:
v

10

In [36]:
initial_population = []
population_size = 100
for i in range(population_size):
    random_member = random_initial_solution(nodes, v)
    initial_population.append(random_member)
p_best_w = 10
g_best_w = 30

In [37]:
def repair_tour_in_solution(solution, index):
    tour = solution[index]
    tour_size = len(tour)
    
    for j in range(len(solution)):
        i = 0
        while i < tour_size:   
            if j != index and tour[i].num in tour_to_num(solution[j]):
                tour.pop(i)

                i -= 1
                tour_size -= 1
            i += 1
    solution[index] = tour
    return solution

def mutation_insert(solution, index):
    candidate = copy.deepcopy(nodes)
    candidate.pop(0)
    for i in range(len(solution)):
        for j in range(len(solution[i])):
                k = 0
                candidate_size = len(candidate)
                while k < candidate_size:
                    if candidate[k].num == solution[i][j].num:
                        candidate.pop(k)
                        break
                    k += 1
    tour = solution[index]
    random.shuffle(candidate)
    can_insert = False
    list_of_index = [i for i in range(len(tour)+1)] 
    random.shuffle(list_of_index)
    for i in random.choices(list_of_index, k=10):
        for j in range(len(candidate)):
            tour.insert(i, candidate[j])
            if is_feasible(tour, source):
                can_insert = True
                break
            else:
                tour.pop(i)
        if can_insert:
            break
    return solution

def mutation_remove(solution, index):
    tour = solution[index]
    if len(tour) <= 1 :
        return solution
    random_index = random.randint(0, len(tour)-1)
    tour.pop(random_index)
    return solution

def mutation_swap(solution, index):
    tour = solution[index]
    can_swap = 0
    for i in range(len(tour)):
        for j in range(i, len(tour)):
            tour[i], tour[j] = tour[j], tour[i]
            if not is_feasible(tour, source):
                tour[i], tour[j] = tour[j], tour[i]
            else:
                can_swap += 1
                break
        if can_swap == 1:
            break
    return solution

def cross_over_and_mutation(first, second, third):
    current_solution = copy.deepcopy(first)
    p_best = copy.deepcopy(second)
    g_best = copy.deepcopy(third)
    for i in range(len(current_solution)):
        uniform_probability = random.uniform(0, 1)
        random_tour_index = random.randint(0, len(current_solution)-1)
        current_solution = mutation_swap(current_solution, random_tour_index)
        if uniform_probability >= 0.2:
            current_solution = mutation_insert(current_solution, random_tour_index)
        else:
            
            current_solution = mutation_remove(current_solution, random_tour_index)

    for i in range(int((p_best_w/100)*v)):
        first_index = random.randint(0, v-1)
        second_index = random.randint(0, v-1)

        current_solution[first_index], p_best[second_index] = p_best[second_index], current_solution[first_index]

        current_solution = repair_tour_in_solution(current_solution, first_index)
        p_best = repair_tour_in_solution(p_best, second_index)

    for i in range(int((g_best_w/100)*v)):
        first_index = random.randint(0, v-1)
        second_index = random.randint(0, v-1)

        current_solution[first_index], g_best[second_index] = g_best[second_index], current_solution[first_index]

        current_solution = repair_tour_in_solution(current_solution, first_index)
        g_best = repair_tour_in_solution(g_best, second_index)


    return current_solution



In [38]:
max_iteration = 5000
g_best = None
g_best_score = 0
for i in range(len(initial_population)):
    solution_score = calculate_score(initial_population[i], source)
    if solution_score > g_best_score:
        g_best_score = solution_score
        g_best = initial_population[i]
p_best = [initial_population[i] for i in range(population_size)]
g_best_score

1190.0

In [39]:

scores = []
population = copy.deepcopy(initial_population)
t_end = time.time() + 60
for i in range(max_iteration):
    if time.time() > t_end:
        break
    new_population = []
    for j in range(population_size):
        p_best_j = p_best[j]
        new_solution = cross_over_and_mutation(population[j], p_best_j, g_best)
        new_solution_score = calculate_score(new_solution, source)
        if new_solution_score >= g_best_score:
            g_best_score = new_solution_score
            g_best = new_solution
        if new_solution_score > calculate_score(p_best_j, source):
            p_best[j] = new_solution
        new_population.append(new_solution)
    population = copy.deepcopy(new_population)
    # pool = new_population + population
    # pool.sort(key=lambda i:calculate_score(i, source), reverse=True)
    # best_score = calculate_score(pool[0], source)
    # best_solution = pool[0]
    # scores.append(best_score)
    # if (i+1) % 10 == 0:
    #     print(i+1, " ", calculate_score(pool[0], source))
    # population = pool[:int(0.7*population_size)] + random.choices(pool[int(0.7*population_size):], k=int(0.3*population_size))


In [40]:
# solution_to_num(population[0])
print(solution_to_num(g_best))
print(calculate_score(g_best, source))

[[13, 15, 29, 73, 50, 86, 89], [18, 58, 91, 52, 59, 7, 2], [40, 22, 14], [93, 61, 23, 72, 32], [46, 63, 98, 84, 95, 4, 51, 54, 31], [28, 33, 64], [3, 25, 53, 11, 1, 66, 97, 20], [85, 70, 39, 43, 94, 81], [92, 57, 88, 12, 34, 37], [60, 75, 62, 99, 69, 16]]
1230.0


In [28]:
for tour in g_best:
    print(is_feasible(tour, source))

True
True
True
True
True
True
True
True
True
True


In [12]:
# import matplotlib.pyplot as plt
# plt.plot([i for i in range(len(scores))],scores)

In [13]:
for tour in g_best:
    each_tour = "0 "
    for node in tour:
        each_tour += str(node.num) + " "
    each_tour += "0"
    print(each_tour)


0 42 54 53 56 58 88 68 52 59 0
0 90 87 84 77 79 80 12 0
0 13 86 27 16 21 82 49 0
0 81 41 40 29 38 6 4 96 85 0
0 33 8 15 94 99 22 72 26 0
0 20 24 25 10 30 39 36 95 69 37 0
0 32 63 62 1 11 97 100 31 34 0
0 43 98 92 7 5 89 3 91 47 0
0 18 74 46 28 83 66 2 48 0
0 67 44 93 64 51 50 75 0


In [14]:
res_line = []
with open("output.txt", 'r') as file:
    for line in file:
        res_line.append(line.rstrip())
res_solution = []
all_res_node = []
for line in res_line:
    tour = []
    for node_id in line.split()[1:-1]:
        tour.append(nodes[int(node_id)])
        all_res_node.append(int(node_id))
    res_solution.append(tour)
if len(all_res_node) != len(set(all_res_node)):
    print("there is duplicate node")
else:
    print("there is no duplicate node")
for i in range(len(res_solution)):
    print(f"tour {i+1} is feasible: {is_feasible(res_solution[i], source)}")
print(f"score: {calculate_score(res_solution, source)}")

there is no duplicate node
tour 1 is feasible: True
tour 2 is feasible: True
tour 3 is feasible: True
tour 4 is feasible: True
tour 5 is feasible: True
tour 6 is feasible: True
tour 7 is feasible: True
tour 8 is feasible: True
tour 9 is feasible: True
tour 10 is feasible: True
score: 1550.0
